In [ ]:
from IPython.core.display import HTML
HTML("""
<style>
@import url('http://fonts.googleapis.com/css?family=Crimson+Text');
@import url('http://fonts.googleapis.com/css?family=Source+Code+Pro');

/* Change code font */
.CodeMirror pre {
    font-family: 'Input Mono Narrow', 'Source Code Pro', Consolas, monocco, monospace;
}

div.input_area {
    border-color: rgba(0,0,0,0.10);
}

div.text_cell {
    max-width: 105ex; /* instead of 100%, */
}

div.text_cell_render {
    font-family: "Crimson Text";
    font-size: 12pt;
    line-height: 145%; /* added for some line spacing of text. */
}

div.text_cell_render h1,
div.text_cell_render h2,
div.text_cell_render h3,
div.text_cell_render h4,
div.text_cell_render h5,
div.text_cell_render h6 {
    font-family: 'Crimson Text';
}

.rendered_html pre,
.rendered_html code {
    font-size: medium;
}

.rendered_html ol {
    list-style:decimal;
    margin: 1em 2em;
}

.prompt.input_prompt {
    color: rgba(0,0,0,0.5);
}

.cell.command_mode.selected {
    border-color: rgba(0,0,0,0.1);
}

.cell.edit_mode.selected {
    border-color: rgba(0,0,0,0.15);
    box-shadow: 0px 0px 5px #f0f0f0;
    -webkit-box-shadow: 0px 0px 5px #f0f0f0;
}

div.output_scroll {
    -webkit-box-shadow: inset 0 2px 8px rgba(0,0,0,0.1);
    box-shadow: inset 0 2px 8px rgba(0,0,0,0.1);
    border-radious: 2px;
}

#menubar .navbar-inner {
    -webkit-box-shadow: none;
    box-shadow: none;
    border-radius: 0;
    border: none;
    font-family: lato;
    font-weight: 400;
}

.navbar-fixed-top .navbar-inner,
.navbar-static-top .navbar-inner {
    box-shadow: none;
    -webkit-box-shadow: none;
    border: none;
}

div#notebook_panel {
    box-shadow: none;
    -webkit-box-shadow: none;
    border-top: none;
}

div#notebook {
    border-top: 1px solid rgba(0,0,0,0.15);
}

/* 
    This is a lazy fix, we *should* fix the 
    background for each Bootstrap button type
*/
#site * .btn {
    -webkit-box-shadow: none;
    box-shadow: none;
}


span.ansiblack {color: #073642;}
span.ansiblue {color: #2aa198;}
span.ansigray {color: #839496;}
span.ansigreen {color: #859900;}
span.ansipurple {color: #6c71c4;}
span.ansired {color: #dc322f;}
span.ansiyellow {color: #b58900;}

div.output_stderr {background-color: #dc322f;}
div.output_stderr pre {color: #eee8d5;}

.cm-s-ipython.CodeMirror {background: #fdf6e3; color: #073642;}
.cm-s-ipython div.CodeMirror-selected {background: #eee8d5 !important;}
.cm-s-ipython .CodeMirror-gutters {background: #fdf6e3; border-right: 0px;}
.cm-s-ipython .CodeMirror-linenumber {color: #839496;}
.cm-s-ipython .CodeMirror-cursor {border-left: 1px solid #657b83 !important;}

.cm-s-ipython span.cm-comment {color: #d33682;}
.cm-s-ipython span.cm-atom {color: #6c71c4;}
.cm-s-ipython span.cm-number {color: #6c71c4;}

.cm-s-ipython span.cm-property, .cm-s-ipython span.cm-attribute {color: #859900;}
.cm-s-ipython span.cm-keyword {color: #dc322f;}
.cm-s-ipython span.cm-string {color: #b58900;}
.cm-s-ipython span.cm-operator {color: #d33682;}
.cm-s-ipython span.cm-builtin {color: #6c71c4;}

.cm-s-ipython span.cm-variable {color: #859900;}
.cm-s-ipython span.cm-variable-2 {color: #268bd2;}
.cm-s-ipython span.cm-def {color: #cb4b16;}
.cm-s-ipython span.cm-error {background: #dc322f; color: #657b83;}
.cm-s-ipython span.cm-bracket {color: #586e75;}
.cm-s-ipython span.cm-tag {color: #dc322f;}
.cm-s-ipython span.cm-link {color: #6c71c4;}

.cm-s-ipython .CodeMirror-matchingbracket { text-decoration: underline; color: #073642 !important;}
th, td { 
      font-size: 18px;
      border-collapse: collapse;
      border-width:3px;
    }
    </style>

""")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from sklearn.preprocessing import RobustScaler, QuantileTransformer
import rasterio
from matplotlib import colors
from scipy import stats
from tqdm import *
from pyproj import Transformer
import ast
import glob
from io import StringIO

| severe_storm_score        | Calculated score based on survey response                                 | CDP Survey               |
|---------------------------|---------------------------------------------------------------------------|--------------------------|
| fema_extreme_events       | # of FEMA severe storm disasters                                          | FEMA National Risk Index |
| percent_built_before_1999 | Percentage of buildings built before 1999                                 | US Census                |
| precip_averaged           | Minimum temp per month averaged over the last 20 years                    | NOAA tmin data           |
| precip_std                | Variability in minimum temp per month averaged over the last 20 years     | NOAA tmin data           |
| precip_dt                 | Rate of change in minimum temp per month averaged over the last 20 years  | NOAA tmin data           |

In [ ]:
path = "../input/externald/Supplementary Data/" 
vulnerability = pd.read_csv(path + "cities_updated_geo_us_2020.csv")
acct_number = vulnerability["Account Number"]
counties = (vulnerability['counties'])
counties_weights = (vulnerability['counties_weights'])
states = (vulnerability['state'])
flag_multi_counties = vulnerability['flag_multiple_counties']
match_fips = vulnerability['FIPS']
MSA = (vulnerability['MSA'])
match_fips = [ast.literal_eval(match_fips[i]) if flag_multi_counties[i] else int(match_fips[i]) for i in range(len(match_fips))]
cities = vulnerability["city_bing"]

In [ ]:
def standardize_rank(arr, direction=1):
    rank = (stats.mstats.rankdata(np.ma.masked_invalid(arr)))
    rank[rank == 0] = np.nan
    if(direction == 1):
        return rank/(np.nanmax(rank)) 
    else:
        return 1-(rank/(np.nanmax(rank)))


In [ ]:
cities_2020 = pd.read_csv("/kaggle/input/cdp-unlocking-climate-solutions/Cities/Cities Responses/2020_Full_Cities_Dataset.csv")

cities_disc_2020 = pd.read_csv("/kaggle/input/cdp-unlocking-climate-solutions/Cities/Cities Disclosing/2020_Cities_Disclosing_to_CDP.csv")

cities_2020_merged = pd.merge(cities_2020, cities_disc_2020, on="Account Number", how="outer", copy=False)
unique_20 = np.unique(cities_disc_2020[cities_disc_2020["Country"] == "United States of America"]["Account Number"])
cities_2020_merged_sub = cities_2020_merged[np.in1d(cities_2020_merged["Account Number"], unique_20)]

def fetch_answer_individual(account, question_number, org_type="city", year=2018, corp_res_type=None, column_number=None, row_number=None):
    if(org_type=="city"):
        df = cities_2020_merged_sub
        subset = df[df['Question Number'] == question_number]
        answer = subset[subset["Account Number"] == account]
        if((column_number is not None) and (row_number is None)):
            answer = answer[answer["Column Number"] == column_number]
        elif((column_number is not None) and (row_number is not None)):
            answer = answer[(answer["Column Number"] == column_number) & (answer["Row Number"] == row_number)]
        elif((column_number is None) and (row_number is not None)):
            answer = answer[answer["Row Number"] == row_number]
        else:
            pass
    elif(org_type=="corp" and corp_res_type=="cc"):
        df = corporations_2020
        subset = df[df['question_number'] == question_number]
        answer = subset[subset["account_number"] == account]
        if((column_number is not None) and (row_number is None)):
            answer = answer[answer["column_number"] == column_number]
        elif((column_number is not None) and (row_number is not None)):
            answer = answer[(answer["column_number"] == column_number) & (answer["row_number"] == row_number)]
        elif((column_number is None) and (row_number is not None)):
            answer = answer[answer["row_number"] == row_number]
        else:
            pass
    elif(org_type=="corp" and corp_res_type=="ws"):
        df = all_corps_cc[year]
        subset = df[df['question_number'] == question_number]
        answer = subset[subset["account_number"] == account]
    else:
        print("Something went wrong. Try again.")
    
    return answer

In [ ]:
scores_lo_hi = {
    "Low" : 1, 
    "Medium Low": 2, 
    "Medium":3,
    "Medium High":4,
    "High":5,
    "Do not know":0.5,
    "Does not currently impact the city":0.5,
    "nan":0,
}

scores_increasing_decreasing = {
    "Increasing" : 2,
    "Decreasing" : 0.5,
    "None" : 1, 
    "Do not know" : 1, 
    "Not expected to happen in the future" : 0.1,
    "nan" : 0.0
}

def score_hazard_risk(cdf):
    prob = scores_lo_hi[str(cdf[cdf["Column Number"] == 3.0]["Response Answer"].iloc[0])]
    mag = scores_lo_hi[str(cdf[cdf["Column Number"] == 4.0]["Response Answer"].iloc[0])]
    
    future_freq = scores_increasing_decreasing[str(cdf[cdf["Column Number"] == 8.0]["Response Answer"].iloc[0])]
    future_intensity = scores_increasing_decreasing[str(cdf[cdf["Column Number"] == 9.0]["Response Answer"].iloc[0])]
    
    future_mag = scores_lo_hi[str(cdf[cdf["Column Number"] == 10.0]["Response Answer"].iloc[0])]
    return (prob*mag) + (prob*future_freq*future_intensity*future_mag)

def return_risk_and_score_per_county(hazard_type):
    risk = []
    self_reported_score = []

    for i in range(len(vulnerability)):
        df = fetch_answer_individual(vulnerability["Account Number"].iloc[i], '2.1')
        idx = np.where(np.array(df["Response Answer"]) == hazard_type)[0]
        if(len(idx) > 0):
            risk.append(1)
            cdf = df[df["Row Number"] == df["Row Number"].iloc[idx[0]]]
            self_reported_score.append(score_hazard_risk(cdf))
        else:
            risk.append(np.nan)
            self_reported_score.append(np.nan)
    return risk, np.nan_to_num(np.array(self_reported_score))


In [ ]:
risk_severe_wind, score_severe_wind = return_risk_and_score_per_county("Storm and wind > Severe wind")

risk_tornado, score_tornado = return_risk_and_score_per_county("Storm and wind > Tornado")

risk_cyclone, score_cyclone = return_risk_and_score_per_county("Storm and wind > Cyclone (Hurricane / Typhoon)")

risk_tropical, score_tropical = return_risk_and_score_per_county("Storm and wind > Tropical storm")

risk_thunderstorm, score_thunderstorm = return_risk_and_score_per_county("Storm and wind > Lightning / thunderstorm")

risk_storm_surge, score_storm_surge = return_risk_and_score_per_county("Storm and wind > Storm surge")


In [ ]:
severe_storm_score = score_severe_wind + score_tornado + score_cyclone + score_tropical + score_thunderstorm + score_storm_surge
severe_storm_ranked = standardize_rank(severe_storm_score, 1)

In [ ]:
fema = pd.read_csv("../input/femadisasters/DisasterDeclarationsSummaries.csv")
ct = np.char.zfill(np.array(fema['fipsCountyCode']).astype(str), 3)
st = np.char.zfill(np.array(fema['fipsStateCode']).astype(str), 2)
fema['fips'] = np.core.defchararray.add(st, ct).astype(int)

In [ ]:
fema_extreme_events = []
for i in range(len(match_fips)):
    if(not flag_multi_counties[i]):
        fips = int(match_fips[i])
        cdf = fema[(fema["incidentType"] == "Typhoon") | (fema["incidentType"] == "Severe Storm(s)") | (fema["incidentType"] == "Tornado")| (fema["incidentType"] == "Hurricane") ]
        cdf = cdf[cdf["fips"] == fips]
        fema_extreme_events.append(len(cdf))
    else:
        fips = match_fips[i]
        temp = []
        for j in range(len(fips)):
            cdf = fema[(fema["incidentType"] == "Typhoon") | (fema["incidentType"] == "Severe Storm(s)") | (fema["incidentType"] == "Tornado")| (fema["incidentType"] == "Hurricane") ]            
            cdf = cdf[cdf["fips"] == fips[j]]
            temp.append(len(cdf))
        ar = np.array(temp)
        weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
        fema_extreme_events.append(weighted_avg)


In [ ]:
fema_extreme_events_ranked = standardize_rank(fema_extreme_events, 1)

In [ ]:
building = pd.read_csv("../input/housingyear/Year_Housing/Building_Year.csv", skiprows=1)
building.head()
building = building.loc[:, ~building.columns.str.startswith('Margin')]
building["id"] = building["id"].str[9:].astype(int)

In [ ]:
percent_built_before_1999 = []
for i in range(len(match_fips)):
    if(not flag_multi_counties[i]):
        fips = int(match_fips[i])
        cdf = building[building["id"] == fips]
        percent_built_before_1999.append(np.sum(np.array(cdf)[0][6:])/(np.array(cdf)[0][2]))
    else:
        fips = match_fips[i]
        temp = []
        for j in range(len(fips)):
            cdf = building[building["id"] == fips[j]]
            temp.append(np.sum(np.array(cdf)[0][6:])/(np.array(cdf)[0][2]))
        ar = np.array(temp)
        weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
        percent_built_before_1999.append(weighted_avg)


In [ ]:
percent_built_before_1999_ranked = standardize_rank(percent_built_before_1999, 1)

In [ ]:
str_precip = open('../input/precipitationyearnoaa/climdiv-pcpncy-v1.0.0-20201104', 'r').read()
columns = ["Code", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
for df in pd.read_fwf(StringIO(str_precip), header=None, chunksize=500000, names=columns,
                      converters={h:str for h in columns}):
    precip = df

years = []
fips = []
for i in range(len(precip)):
    fips.append(int(str(precip["Code"].iloc[i])[0:5]))
    years.append(int(str(precip["Code"].iloc[i])[7:11]))
    
precip["year"] = np.array(years).astype(int)
precip["fips"] = np.array(fips).astype(int)

state_fips = pd.read_csv("../input/statefips/state_fips.csv")
df_states = state_fips.drop([1,10])
df_states = df_states.reset_index()
df_states["index"] = np.array(df_states.index) + 1
fips_to_code = dict(zip(df_states["fips"], df_states["index"]))
fips_to_code[15] = 99 # HI
fips_to_code[11] = 99 # DC
fips_to_code[2] = 99 # AK

precip_max_averaged = []
for i in range(len(match_fips)):
    if(not flag_multi_counties[i]):
        fips = int(match_fips[i])
        padstr = str(np.char.zfill(str(fips), width=5))
        st_code = int((padstr)[0:2])
        fips = int(str(fips_to_code[st_code]) + padstr[2:])
        cdf = precip[(precip['fips'] == fips) & (precip['year'] > 2000)]
        cdf = cdf.drop(columns=["Code", "year", "fips"]).astype(np.float)
        cdf = cdf[(cdf != -9.99)]
        precip_max_averaged.append(np.mean(np.max(cdf, axis=1)))
    else:
        fips = match_fips[i]
        temp = []
        for j in range(len(fips)):
            padstr = str(np.char.zfill(str(fips[j]), width=5))
            st_code = int((padstr)[0:2])
            tfips = int(str(fips_to_code[st_code]) + padstr[2:])
            cdf = precip[(precip['fips'] == tfips) & (precip['year'] > 2000)]
            cdf = cdf.drop(columns=["Code", "year", "fips"]).astype(np.float)     
            cdf = cdf[(cdf != -9.99)]
            temp.append(np.mean(np.max(cdf, axis=1)))
        ar = np.array(temp)
        weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
        precip_max_averaged.append(weighted_avg)

precip_max_std = []
for i in range(len(match_fips)):
    if(not flag_multi_counties[i]):
        fips = int(match_fips[i])
        padstr = str(np.char.zfill(str(fips), width=5))
        st_code = int((padstr)[0:2])
        fips = int(str(fips_to_code[st_code]) + padstr[2:])
        cdf = precip[(precip['fips'] == fips) & (precip['year'] > 2000)]
        cdf = cdf.drop(columns=["Code", "year", "fips"]).astype(np.float)  
        cdf = cdf[(cdf != -9.99)]
        precip_max_std.append(np.std(np.max(cdf, axis=1)))
    else:
        fips = match_fips[i]
        temp = []
        for j in range(len(fips)):
            padstr = str(np.char.zfill(str(fips[j]), width=5))
            st_code = int((padstr)[0:2])
            tfips = int(str(fips_to_code[st_code]) + padstr[2:])
            cdf = precip[(precip['fips'] == tfips) & (precip['year'] > 2000)]
            cdf = cdf.drop(columns=["Code", "year", "fips"]).astype(np.float)     
            cdf = cdf[(cdf != -9.99)]
            temp.append(np.mean(np.std(cdf, axis=1)))
        ar = np.array(temp)
        weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
        precip_max_std.append(weighted_avg)

precip_max_dt = []
for i in range(len(match_fips)):
    if(not flag_multi_counties[i]):
        fips = int(match_fips[i])
        padstr = str(np.char.zfill(str(fips), width=5))
        st_code = int((padstr)[0:2])
        fips = int(str(fips_to_code[st_code]) + padstr[2:])
        cdf = precip[(precip['fips'] == fips) & (precip['year'] > 1980)]
        cdf = cdf.drop(columns=["Code", "year", "fips"]).astype(np.float)  
        cdf = cdf[(cdf != -9.99)]
        precip_max_dt.append(np.median(np.diff(np.max(cdf, axis=1))))
    else:
        fips = match_fips[i]
        temp = []
        for j in range(len(fips)):
            padstr = str(np.char.zfill(str(fips[j]), width=5))
            st_code = int((padstr)[0:2])
            tfips = int(str(fips_to_code[st_code]) + padstr[2:])
            cdf = precip[(precip['fips'] == tfips) & (precip['year'] > 1980)]
            cdf = cdf.drop(columns=["Code", "year", "fips"]).astype(np.float)     
            cdf = cdf[(cdf != -9.99)]
            temp.append(np.median(np.diff(np.max(cdf, axis=1))))
        ar = np.array(temp)
        weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
        precip_max_dt.append(weighted_avg)

precip_max_averaged_ranked = standardize_rank(precip_max_averaged, 1)
precip_max_std_ranked = standardize_rank(precip_max_std, 1)
precip_max_dt_ranked = standardize_rank(precip_max_dt, 1)

In [ ]:
nri = pd.read_csv(path + "NRI_Table_Counties.csv")

def nri_select(field):
    res = []
    for i in range(len(match_fips)):
        if(not flag_multi_counties[i]):
            fips = int(match_fips[i])
            cdf = nri[nri["STCOFIPS"] == fips]
            try:
                res.append(float(cdf[field]))
            except:
                res.append(np.nan)
        else:
            fips = match_fips[i]
            temp = []
            for j in range(len(fips)):
                cdf = nri[nri["STCOFIPS"] == fips[j]]
                try:
                    temp.append(float(cdf[field]))
                except:
                    temp.append(np.nan)
            ar = np.array(temp)
            weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
            res.append(weighted_avg)
    return np.array(res)



In [ ]:
avalanche_eals = nri_select("AVLN_EALS")
earthquake_eals = nri_select("ERQK_EALS")
strong_wind_eals = nri_select("SWND_EALS")
tornado_eals = nri_select("TRND_EALS")

avalanche_eals_ranked = standardize_rank(avalanche_eals, 1)
earthquake_eals_ranked = standardize_rank(earthquake_eals, 1)
strong_wind_eals_ranked = standardize_rank(strong_wind_eals, 1)
tornado_eals_ranked = standardize_rank(tornado_eals, 1)

In [ ]:
vulnerability_raw = vulnerability.copy()
vulnerability_ranked = vulnerability.copy()
vulnerability_aggr = vulnerability.copy()

In [ ]:
### EXPOSURE 

vulnerability_raw["score_severe_storm"] = severe_storm_score
vulnerability_raw["fema_extreme_events"] = fema_extreme_events
vulnerability_raw["precip_max_averaged"] = precip_max_averaged
vulnerability_raw["precip_max_std"] = precip_max_std
vulnerability_raw["precip_max_dt"] = precip_max_dt

### ECONOMIC
vulnerability_raw["percent_built_before_1999"] = percent_built_before_1999
vulnerability_raw["avalanche_expected_annual_loss"] = avalanche_eals

vulnerability_raw["earthquake_expected_annual_loss"] = earthquake_eals
vulnerability_raw["strong_wind_expected_annual_loss"] = strong_wind_eals
vulnerability_raw["tornado_expected_annual_loss"] = tornado_eals

vulnerability_raw.to_csv("vulnerability_cities_us_severe_storms_kpis_raw.csv")

In [ ]:
### EXPOSURE 

vulnerability_ranked["severe_storm_ranked"] = severe_storm_ranked
vulnerability_ranked["fema_extreme_events_ranked"] = fema_extreme_events_ranked
vulnerability_ranked["precip_max_averaged_ranked"] = precip_max_averaged_ranked
vulnerability_ranked["precip_max_std_ranked"] = precip_max_std_ranked
vulnerability_ranked["precip_max_dt_ranked"] = precip_max_dt_ranked


### ECONOMIC
vulnerability_ranked["percent_built_before_1999_ranked"] = percent_built_before_1999_ranked
vulnerability_ranked["avalanche_expected_annual_loss_ranked"] = avalanche_eals_ranked

vulnerability_ranked["earthquake_expected_annual_loss_ranked"] = earthquake_eals_ranked
vulnerability_ranked["strong_wind_expected_annual_loss_ranked"] = strong_wind_eals_ranked
vulnerability_ranked["tornado_expected_annual_loss_ranked"] = tornado_eals_ranked

vulnerability_ranked.to_csv("vulnerability_cities_us_severe_storms_kpis_ranked.csv")

In [ ]:
def aggr_and_rescale(arr):
    avg_arr = np.nanmean(arr, axis=1)
    rescaled_avg_arr = standardize_rank(avg_arr)
    return rescaled_avg_arr

In [ ]:
all_expo_severe_kpis = np.c_[severe_storm_ranked, fema_extreme_events_ranked, precip_max_averaged_ranked, precip_max_std_ranked,
                          precip_max_dt_ranked]

aggr_expo_severe_kpis = aggr_and_rescale(all_expo_severe_kpis)

all_sens_severe_kpis = np.c_[percent_built_before_1999_ranked, avalanche_eals_ranked, earthquake_eals_ranked, strong_wind_eals_ranked, 
                           tornado_eals_ranked]

aggr_sens_severe_kpis = aggr_and_rescale(all_sens_severe_kpis)

aggr_severe_kpi = aggr_and_rescale(np.c_[aggr_expo_severe_kpis, aggr_sens_severe_kpis])

In [ ]:
vulnerability_aggr = vulnerability.copy()

vulnerability_aggr["aggr_severe_kpis"] = aggr_severe_kpi
vulnerability_aggr["aggr_exposure_severe_kpis"] = aggr_expo_severe_kpis
vulnerability_aggr["aggr_sensitivity_severe_kpis"] = aggr_sens_severe_kpis

vulnerability_aggr.to_csv("vulnerability_cities_us_severe_aggr_kpis.csv")